In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 45.8MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12859254969648030877
xla_global_id: -1
]


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [12, 13]
BiLSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in BiLSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(BiLSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))


                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'BiLSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'BiLSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_13",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.059350751476846764,
  "MSE_normalize": 0.004116669712347437,
  "RMSE_normalize": 0.06416127891764188,
  "MAPE_normalize": 0.06234711067901449,
  "MAE": 1074.0122767857142,
  "MSE": 1348068.737422398,
  "RMSE": 1161.0636233309517,
  "MAPE": 0.02653660652844252
}
Percent CPU Usage: 7.3
Percent Ram Usage: 2.9
2022-10-12 10:37:31.894549+07:00
PROGRESS: [ 2 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_13",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 15,
  "patience_list": 3,
  "MAE_normalize": 0.05008884532984359,
  "MSE_normalize": 0.0029786442569169694,
  "RMSE_normalize": 0.05457695719730965,
  "MAPE_normalize": 0.05267780944182111,
  "MAE": 906

{
  "model": "BiLSTM",
  "AHB_number": "AHB_13",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 18,
  "patience_list": 3,
  "MAE_normalize": 0.19601787533952203,
  "MSE_normalize": 0.039233460081023276,
  "RMSE_normalize": 0.1980743801732654,
  "MAPE_normalize": 0.207725745780649,
  "MAE": 3547.1395089285716,
  "MSE": 12847592.73437936,
  "RMSE": 3584.3538796245216,
  "MAPE": 0.08796851098342416
}
Percent CPU Usage: 16.3
Percent Ram Usage: 3.4
2022-10-12 10:37:56.124577+07:00
PROGRESS: [ 6 / 5040 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 59459.42857142857,
  "MSE": 3584359358.651786,
  "RMSE": 59869.51944563933,
  "MAPE": 0.04852804868772907
}
Percent CPU Usage: 22.9
Percent Ram Usage: 3.5
2022-10-13 02:02:09.049723+07:00
PROGRESS: [ 4833 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "Country_Level",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 50,
  "patience_list": 10,
  "MAE_normalize": 0.09181732490063739,
  "MSE_normalize": 0.008484584127342104,
  "RMSE_normalize": 0.0921118023238179,
  "MAPE_normalize": 0.09567493688389861,
  "MAE": 59037.80357142857,
  "MSE": 3507854975.863839,
  "RMSE": 59227.147287910455,
  "MAPE": 0.04820957416821852
}
Percent CPU Usage: 21.1
Percent Ram Usage: 3.5
2022-10-13 02:02:16.546624+07:00
PROGRESS: [ 4834 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "Country_Level",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  

In [ ]:
name = "all_performance_BiLSTM_5nsteps_AHB13-C"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_BiLSTM_5nsteps_AHB13-C.json /content/drive/MyDrive/Depression
! cp /content/all_performance_BiLSTM_5nsteps_AHB13-C.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,BiLSTM_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,BiLSTM,AHB_13,uni,1,8,0.2,2,5,12,3,0.059351,0.004117,0.064161,0.062347,1074.012277,1.348069e+06,1161.063623,0.026537
1,BiLSTM,AHB_13,mul,1,8,0.2,2,5,15,3,0.050089,0.002979,0.054577,0.052678,906.407366,9.754018e+05,987.624345,0.022406
2,BiLSTM,AHB_13,uni,1,8,0.2,4,5,16,3,0.071907,0.005811,0.076229,0.075654,1301.220424,1.902876e+06,1379.447693,0.032172
3,BiLSTM,AHB_13,mul,1,8,0.2,4,5,23,3,0.060622,0.004083,0.063902,0.063925,1097.010603,1.337176e+06,1156.363356,0.027149
4,BiLSTM,AHB_13,uni,1,8,0.2,8,5,18,3,0.196018,0.039233,0.198074,0.207726,3547.139509,1.284759e+07,3584.353880,0.087969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,BiLSTM,Country_Level,mul,3,128,0.5,32,5,27,10,0.029059,0.000892,0.029874,0.030146,18684.642857,3.689854e+08,19208.993102,0.015224
5036,BiLSTM,Country_Level,uni,3,128,0.5,64,5,73,10,0.038589,0.001510,0.038861,0.040171,24812.178571,6.243743e+08,24987.482996,0.020252
5037,BiLSTM,Country_Level,mul,3,128,0.5,64,5,23,10,0.184285,0.034102,0.184667,0.192108,118493.767857,1.409910e+10,118739.629037,0.096781
5038,BiLSTM,Country_Level,uni,3,128,0.5,128,5,20,10,0.067431,0.004566,0.067575,0.070544,43357.535714,1.887921e+09,43450.212864,0.035476
